In [14]:
import openrouteservice
from bs4 import BeautifulSoup
import requests
import unicodedata
from csv import writer
import re
import pandas as pd
import numpy as np
import json


#spark session
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import functions as F


import os
import re 
import time

#folium
import geopandas as gpd
import re
import folium

In [15]:

output_dir = '../../data/raw/ors-data/'





In [16]:
# use your own key
client = openrouteservice.Client(key='your_key') # Provide your personal API key

In [17]:
# read files
domain = pd.read_csv('../../data/curated/domain-website-data/domain_preprocessed.csv')

# external datasets
hospitals = pd.read_csv('../../data/raw/external-data/hospital.csv')
care_facility = pd.read_csv('../../data/raw/external-data/care_facility.csv')
emergency = pd.read_csv('../../data/raw/external-data/emergency_service.csv')


# file with postcode centroid coordinates
centroid = pd.read_csv('../../data/raw/external-data/postcode_centroid.csv')


In [18]:
#postcode with properties
postcode_list = list(domain['postcode'].unique())

# remove external data not in domain postcode list
hospitals= hospitals.loc[hospitals['postcode'].isin(postcode_list)].reset_index().drop(columns = ['Unnamed: 0'])
care_facility= care_facility.loc[care_facility['postcode'].isin(postcode_list)].reset_index().drop(columns = ['Unnamed: 0'])
emergency= emergency.loc[emergency['postcode'].isin(postcode_list)].reset_index().drop(columns = ['Unnamed: 0'])

In [6]:
"""


hospital


"""

dic_hospital = {'distance': [], 'duration': []}
count = 0

for i in range(len(hospitals)):
    if count/10 == 0:
            time.sleep(5)
    current_postcode = hospitals['postcode'][i]
    txt = hospitals['geometry'][i]
    current_point_txt = re.search("\((.*?)\)",txt)[0]
    current_point = eval(current_point_txt.replace(" ", ","))
    postcode_centroid = eval(centroid[centroid.POSTCODE == current_postcode]['centroid'].values[0])
    route = client.directions(
                coordinates= [postcode_centroid, current_point],
                profile='driving-car',
                format='geojson',
                radiuses = 1000,
            )
    distance = route['features'][0]['properties']['segments'][0]['distance']
    duration = route['features'][0]['properties']['segments'][0]['duration']
    dic_hospital['distance'].append(distance)
    dic_hospital['duration'].append(duration)
    count += 1

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate l

In [8]:
# add distance and duration column to hospital df
hospitals['distance'] = dic_hospital['distance']
hospitals['durations'] = dic_hospital['duration']

In [33]:
# choose hopistal with shortest duration by postcode
code_selected = list(hospitals.groupby('postcode').min('duration')['code'])
hospitals_filtered= hospitals[hospitals.code.isin(code_selected)]

In [34]:
# output directory
output_dir = '../../data/raw/ors-data/'

# output file
filename = 'hospital_by_postcode.csv'
output_dir_full = f'{output_dir}{filename}'
hospitals_filtered.to_csv(output_dir_full)



In [19]:
'''


emergency


'''

# create dictionary to store
emergency_facility = {'distance': [], 'duration': []}
count = 0

for i in range(len(emergency)):
    if count/5 == 0:
            time.sleep(10)
            
    current_postcode = emergency['postcode'][i]
    # geometry location coordinates in string
    txt = emergency['geometry'][i]
    #find the coordinates
    current_point_txt = re.search("\((.*?)\)",txt)[0]
    # convert to tuple
    current_point = eval(current_point_txt.replace(" ", ","))
    postcode_centroid = eval(centroid[centroid.POSTCODE == current_postcode]['centroid'].values[0])
    route = client.directions(
                coordinates= [postcode_centroid, current_point],
                profile='driving-car',
                format='geojson',
                radiuses = 1000,
            )
    distance = route['features'][0]['properties']['segments'][0]['distance']
    duration = route['features'][0]['properties']['segments'][0]['duration']
    emergency_facility['distance'].append(distance)
    emergency_facility['duration'].append(duration)
    count += 1

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate l

In [20]:
# add distance and duration column to care facility df
emergency['distance'] = emergency_facility['distance']
emergency['durations'] = emergency_facility['duration']


In [25]:
emergency = emergency.drop(columns = ['index']).reset_index()

In [28]:
# choose care_facility with shortest duration by postcode
code_selected = list(emergency.groupby('postcode').min('duration')['index'])
emergency_filtered = emergency[emergency.index.isin(code_selected)]

In [32]:
# output directory
output_dir = '../../data/raw/ors-data/'

# output
filename = 'emergency_by_postcode.csv'
output_dir_full = f'{output_dir}{filename}'
emergency_filtered.to_csv(output_dir_full)


